# Undersampling & Oversampling

Undersampling aims to decrease the number of instances from the overrepresented class in a data set, they are said to be unbalanced. Unbalanced refers to highly unequal or uneven distribution of classes. 

In [31]:
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE #Using nearest neighbors algorithm to generate new and synthetic data
from sklearn.model_selection import train_test_split

from preprocessing_ml import *

In [20]:
dataframe = pd.read_csv("framingham.csv") #Importing the data

In [21]:
###Seeing the difference between CHD after 10 years and no-CHD.

CHD_count = dataframe.TenYearCHD.value_counts() #Shows the count of no-CHD and CHD after 10years
print('The CHD after ten years has a ratio of approximately {} to 1.'.format(int(CHD_count[0] / CHD_count[1])))

The CHD after ten years has a ratio of approximately 5 to 1.


In [32]:
y = dataframe['TenYearCHD']
X = dataframe.drop('TenYearCHD', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)